In [1]:
import random

In [2]:
class Sentiment: 
    NEGATIVE = "NEGATIVE"
    NEUTRAL = "NEUTRAL"
    POSITIVE = "POSITIVE"
    
class Review: 
    def __init__(self, text, score):
        self.text = text 
        self.score = score
        self.sentiment = self.get_sentiment()
        
    def get_sentiment(self):
        if self.score <= 2: return Sentiment.NEGATIVE
        elif self.score == 3:  return Sentiment.NEUTRAL
        else: return Sentiment.NEGATIVE #Score of 4 or 5

class ReviewContainer: 
    def __init__(self, reviews):
        self.reviews = reviews
    
    def get_text(self):
        return [review.text for review in self.reviews]
    
    def get_sentiment(self):
        return [review.sentiment for review in self.reviews]
    
    def evenly_distrinute(self): 
        positive = list(filter(lambda review: review.sentiment == Sentiment.POSITIVE, self.reviews))
        negative = list(filter(lambda review: review.sentiment == Sentiment.NEGATIVE, self.reviews))
        positive_shrunk = positive[:len(negative)]
        self.reviews = negative + positive_shrunk
        random.shuffle(self.reviews)

In [3]:
import json

file_name = './data/books_small_10000.json'
reviews = []

with open(file_name) as f: 
    for line in f:
        review = json.loads(line)
        reviews.append(Review(review['reviewText'], review['overall']))

reviews[5].text 

'I hoped for Mia to have some peace in this book, but her story is so real and raw.  Broken World was so touching and emotional because you go from Mia\'s trauma to her trying to cope.  I love the way the story displays how there is no "just bouncing back" from being sexually assaulted.  Mia showed us how those demons come for you every day and how sometimes they best you. I was so in the moment with Broken World and hurt with Mia because she was surrounded by people but so alone and I understood her feelings.  I found myself wishing I could give her some of my courage and strength or even just to be there for her.  Thank you Lizzy for putting a great character\'s voice on a strong subject and making it so that other peoples story may be heard through Mia\'s.'

In [4]:
from sklearn.model_selection import train_test_split

training, test = train_test_split(reviews, test_size=0.33, random_state=42)
train_container = ReviewContainer(training)
test_container = ReviewContainer(test)

In [ ]:
train_container.evenly_distribute()
train_x = train_container 